# 論文 12：用於量子化學的神經訊息傳遞
## Justin Gilmer, Samuel S. Schoenholz, Patrick F. Riley, Oriol Vinyals, George E. Dahl (2017)

### 訊息傳遞神經網路 (MPNNs)

圖神經網路的統一框架。現代 GNN 的基礎！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

np.random.seed(42)

## 圖表示

In [ ]:
class Graph:
    """簡單的圖表示"""
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes
        self.edges = []  # (source, target) 元組列表
        self.node_features = []  # 節點特徵向量列表
        self.edge_features = {}  # 字典：(src, tgt) -> 邊特徵
    
    def add_edge(self, src, tgt, features=None):
        self.edges.append((src, tgt))
        if features is not None:
            self.edge_features[(src, tgt)] = features
    
    def set_node_features(self, features):
        """features：特徵向量列表"""
        self.node_features = features
    
    def get_neighbors(self, node):
        """獲取節點的所有鄰居"""
        neighbors = []
        for src, tgt in self.edges:
            if src == node:
                neighbors.append(tgt)
        return neighbors
    
    def visualize(self, node_labels=None):
        """使用 networkx 視覺化圖"""
        G = nx.DiGraph()
        G.add_nodes_from(range(self.num_nodes))
        G.add_edges_from(self.edges)
        
        pos = nx.spring_layout(G, seed=42)
        
        plt.figure(figsize=(10, 8))
        nx.draw(G, pos, with_labels=True, node_color='lightblue', 
               node_size=800, font_size=12, arrows=True,
               arrowsize=20, edge_color='gray', width=2)
        
        if node_labels:
            nx.draw_networkx_labels(G, pos, node_labels, font_size=10)
        
        plt.title("圖結構")
        plt.axis('off')
        plt.show()

# 建立範例分子圖
# H2O（水）：O 連接到 2 個 H 原子
water = Graph(num_nodes=3)
water.add_edge(0, 1)  # O -> H
water.add_edge(0, 2)  # O -> H  
water.add_edge(1, 0)  # H -> O（無向）
water.add_edge(2, 0)  # H -> O

# 節點特徵：[原子序數, 價態, ...]
water.set_node_features([
    np.array([8, 2]),  # 氧
    np.array([1, 1]),  # 氫
    np.array([1, 1]),  # 氫
])

labels = {0: 'O', 1: 'H', 2: 'H'}
water.visualize(labels)

print(f"節點數：{water.num_nodes}")
print(f"邊數：{len(water.edges)}")
print(f"節點 0（氧）的鄰居：{water.get_neighbors(0)}")

## 訊息傳遞框架

**兩個階段：**
1. **訊息傳遞**：從鄰居聚合資訊（T 步）
2. **讀出**：全圖表示

$$m_v^{t+1} = \sum_{w \in N(v)} M_t(h_v^t, h_w^t, e_{vw})$$
$$h_v^{t+1} = U_t(h_v^t, m_v^{t+1})$$

In [ ]:
class MessagePassingLayer:
    """單一訊息傳遞層"""
    def __init__(self, node_dim, edge_dim, hidden_dim):
        self.node_dim = node_dim
        self.edge_dim = edge_dim
        self.hidden_dim = hidden_dim
        
        # 訊息函數：M(h_v, h_w, e_vw)
        self.W_msg = np.random.randn(hidden_dim, 2*node_dim + edge_dim) * 0.01
        self.b_msg = np.zeros(hidden_dim)
        
        # 更新函數：U(h_v, m_v)
        self.W_update = np.random.randn(node_dim, node_dim + hidden_dim) * 0.01
        self.b_update = np.zeros(node_dim)
    
    def message(self, h_source, h_target, e_features):
        """計算從源到目標的訊息"""
        # 串接源、目標、邊特徵
        if e_features is None:
            e_features = np.zeros(self.edge_dim)
        
        concat = np.concatenate([h_source, h_target, e_features])
        
        # 應用訊息網路
        message = np.tanh(np.dot(self.W_msg, concat) + self.b_msg)
        return message
    
    def aggregate(self, messages):
        """聚合訊息（求和）"""
        if len(messages) == 0:
            return np.zeros(self.hidden_dim)
        return np.sum(messages, axis=0)
    
    def update(self, h_node, aggregated_message):
        """更新節點表示"""
        concat = np.concatenate([h_node, aggregated_message])
        h_new = np.tanh(np.dot(self.W_update, concat) + self.b_update)
        return h_new
    
    def forward(self, graph, node_states):
        """
        一次訊息傳遞步驟
        
        graph：Graph 物件
        node_states：當前節點隱藏狀態列表
        
        返回：更新後的節點狀態
        """
        new_states = []
        
        for v in range(graph.num_nodes):
            # 從鄰居收集訊息
            messages = []
            for w in graph.get_neighbors(v):
                # 獲取邊特徵
                edge_feat = graph.edge_features.get((w, v), None)
                
                # 計算訊息
                msg = self.message(node_states[w], node_states[v], edge_feat)
                messages.append(msg)
            
            # 聚合訊息
            aggregated = self.aggregate(messages)
            
            # 更新節點狀態
            h_new = self.update(node_states[v], aggregated)
            new_states.append(h_new)
        
        return new_states

# 測試訊息傳遞
node_dim = 4
edge_dim = 2
hidden_dim = 8

mp_layer = MessagePassingLayer(node_dim, edge_dim, hidden_dim)

# 從特徵初始化節點狀態
initial_states = []
for feat in water.node_features:
    # 嵌入到更高維度
    state = np.concatenate([feat, np.zeros(node_dim - len(feat))])
    initial_states.append(state)

# 執行訊息傳遞
updated_states = mp_layer.forward(water, initial_states)

print(f"\n初始狀態（O）：{initial_states[0]}")
print(f"更新狀態（O）：{updated_states[0]}")
print(f"\n節點狀態通過鄰居資訊更新！")

## 完整 MPNN

In [ ]:
class MPNN:
    """訊息傳遞神經網路"""
    def __init__(self, node_feat_dim, edge_feat_dim, hidden_dim, num_layers, output_dim):
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 嵌入層
        self.embed_W = np.random.randn(hidden_dim, node_feat_dim) * 0.01
        
        # 訊息傳遞層
        self.mp_layers = [
            MessagePassingLayer(hidden_dim, edge_feat_dim, hidden_dim*2)
            for _ in range(num_layers)
        ]
        
        # 讀出（圖級預測）
        self.readout_W = np.random.randn(output_dim, hidden_dim) * 0.01
        self.readout_b = np.zeros(output_dim)
    
    def forward(self, graph):
        """
        通過 MPNN 的前向傳遞
        
        返回：圖級預測
        """
        # 嵌入節點特徵
        node_states = []
        for feat in graph.node_features:
            embedded = np.tanh(np.dot(self.embed_W, feat))
            node_states.append(embedded)
        
        # 訊息傳遞
        states_history = [node_states]
        for layer in self.mp_layers:
            node_states = layer.forward(graph, node_states)
            states_history.append(node_states)
        
        # 讀出：聚合節點狀態為圖表示
        graph_repr = np.sum(node_states, axis=0)  # 簡單求和池化
        
        # 最終預測
        output = np.dot(self.readout_W, graph_repr) + self.readout_b
        
        return output, states_history

# 建立 MPNN
mpnn = MPNN(
    node_feat_dim=2,
    edge_feat_dim=2,
    hidden_dim=8,
    num_layers=3,
    output_dim=1  # 預測單一屬性（例如能量）
)

# 前向傳遞
prediction, history = mpnn.forward(water)

print(f"圖級預測：{prediction}")
print(f"（例如，分子屬性如能量、溶解度等）")

## 視覺化訊息傳遞

In [ ]:
# 視覺化節點表示如何演化
fig, axes = plt.subplots(1, len(history), figsize=(16, 4))

for step, states in enumerate(history):
    # 堆疊節點狀態以供視覺化
    states_matrix = np.array(states).T  # (hidden_dim, num_nodes)
    
    ax = axes[step]
    im = ax.imshow(states_matrix, cmap='RdBu', aspect='auto')
    ax.set_title(f'步驟 {step}')
    ax.set_xlabel('節點')
    ax.set_ylabel('隱藏維度')
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['O', 'H', 'H'])

plt.colorbar(im, ax=axes, label='激活值')
plt.suptitle('訊息傳遞過程中的節點表示', fontsize=14)
plt.tight_layout()
plt.show()

print("\n節點通過聚合鄰居資訊來更新其表示")

## 建立更複雜的圖

In [ ]:
# 建立苯環（C6H6）
benzene = Graph(num_nodes=12)  # 6 個 C + 6 個 H

# 碳環（節點 0-5）
for i in range(6):
    next_i = (i + 1) % 6
    benzene.add_edge(i, next_i)
    benzene.add_edge(next_i, i)

# 氫原子（節點 6-11）連接到碳
for i in range(6):
    h_idx = 6 + i
    benzene.add_edge(i, h_idx)
    benzene.add_edge(h_idx, i)

# 節點特徵
features = []
for i in range(6):
    features.append(np.array([6, 3]))  # 碳
for i in range(6):
    features.append(np.array([1, 1]))  # 氫
benzene.set_node_features(features)

# 視覺化
labels = {i: 'C' for i in range(6)}
labels.update({i: 'H' for i in range(6, 12)})
benzene.visualize(labels)

# 執行 MPNN
pred_benzene, hist_benzene = mpnn.forward(benzene)
print(f"\n苯預測：{pred_benzene}")

## 不同的聚合函數

In [ ]:
# 比較聚合策略
def sum_aggregation(messages):
    return np.sum(messages, axis=0) if len(messages) > 0 else np.zeros_like(messages[0])

def mean_aggregation(messages):
    return np.mean(messages, axis=0) if len(messages) > 0 else np.zeros_like(messages[0])

def max_aggregation(messages):
    return np.max(messages, axis=0) if len(messages) > 0 else np.zeros_like(messages[0])

# 在隨機訊息上測試
test_messages = [np.random.randn(8) for _ in range(3)]

print("聚合函數：")
print(f"求和：{sum_aggregation(test_messages)[:4]}...")
print(f"平均：{mean_aggregation(test_messages)[:4]}...")
print(f"最大：{max_aggregation(test_messages)[:4]}...")
print("\n不同的聚合方式捕捉不同的模式！")

## 關鍵要點

### 訊息傳遞框架：

**階段 1：訊息傳遞**（重複 T 次）
```
對每個節點 v：
  1. 從鄰居收集訊息：
     m_v = Σ_{u∈N(v)} M_t(h_v, h_u, e_uv)
  
  2. 更新節點狀態：
     h_v = U_t(h_v, m_v)
```

**階段 2：讀出**
```
圖表示：
  h_G = R({h_v | v ∈ G})
```

### 元件：
1. **訊息函數 M**：計算來自鄰居的訊息
2. **聚合**：組合訊息（sum、mean、max、attention）
3. **更新函數 U**：更新節點表示
4. **讀出 R**：圖級池化

### 變體：
- **GCN**：簡化的訊息傳遞加正規化
- **GraphSAGE**：鄰居取樣，歸納學習
- **GAT**：基於注意力的聚合
- **GIN**：強大的聚合（求和 + MLP）

### 應用：
- **分子屬性預測**：QM9、藥物發現
- **社交網路**：節點分類、連結預測
- **知識圖譜**：推理、補全
- **推薦系統**：使用者-物品圖
- **3D 視覺**：點雲、網格

### 優勢：
- ✅ 處理可變大小的圖
- ✅ 排列不變性
- ✅ 歸納學習（可泛化到新圖）
- ✅ 可解釋性（訊息傳遞）

### 挑戰：
- 過度平滑（深層使節點變得相似）
- 表達能力（受聚合限制）
- 可擴展性（大圖）

### 現代擴展：
- **圖 Transformer**：對完整圖的注意力
- **等變 GNN**：尊重對稱性（E(3)、SE(3)）
- **時間 GNN**：動態圖
- **異質 GNN**：多種節點/邊類型